In [1]:
import pandas as pd
# import dummy classifier
from sklearn.dummy import DummyClassifier
# import train_test_split
from sklearn.model_selection import train_test_split

In [2]:
data = pd.read_csv('train.csv')

In [3]:
data.head()

,id,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
0,66728,21770,1,156,64.0,140,80,2,1,0,0,1,1
1,69098,21876,1,170,85.0,160,90,1,1,0,0,1,1
2,59185,23270,1,151,90.0,130,80,1,1,0,0,1,1
3,49288,19741,1,159,97.0,120,80,1,1,0,0,1,1
4,62481,18395,1,164,68.0,120,80,1,1,0,0,1,0


In [4]:
x_train, x_val, y_train, y_val = train_test_split(data.drop('cardio', axis=1), data['cardio'], test_size=0.2, random_state=0)

In [5]:
dummy_clf = DummyClassifier(strategy='most_frequent')
dummy_clf.fit(x_train, y_train)

dummy_clf.score(x_val, y_val)

0.4988392857142857

In [6]:
#Naive-bayes algorithm
from sklearn.naive_bayes import GaussianNB

nb_clf = GaussianNB()
nb_clf.fit(x_train,y_train)

nb_clf.score(x_val,y_val)

0.5583035714285715

In [7]:
#import min max scaler (normalizar os dados), one hot encoder (transformar em binários os dados categóricos), and column transformer
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer

# pode deixar dessa forma, se tiver mt ruim a gente muda
var_to_be_used = ['gluc', 'cholesterol', 'ap_hi', 'age', 'ap_lo', 'weight']
numerical_features = ['ap_hi', 'ap_lo', 'age', 'weight']
categorical_features = ['gluc', 'cholesterol']

# create column transformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', MinMaxScaler(), numerical_features),
        ('cat', OneHotEncoder(), categorical_features)
    ])

x_train = preprocessor.fit_transform(x_train)
x_val = preprocessor.transform(x_val)

In [9]:
# import ransearchcv
from sklearn.model_selection import RandomizedSearchCV
# define the parameters
params = {
    'var_smoothing' : [1e-9, 1e-8, 1e-7,1e-6, 1e-5]
}

#create the model
nb_clf = GaussianNB()

# create the random search cv
random_search = RandomizedSearchCV(nb_clf, param_distributions = params, n_iter = 5, scoring = 'roc_auc', n_jobs = -1, cv = 5, verbose = 3)

# fit the model
random_search.fit(x_train, y_train)

random_search.best_params_

Fitting 5 folds for each of 5 candidates, totalling 25 fits


{'var_smoothing': 1e-09}

In [10]:
# train nb with the best params
nb_clf = GaussianNB(**random_search.best_params_)
nb_clf.fit(x_train, y_train)

GaussianNB()

In [11]:
nb_clf.score(x_val,y_val)

0.5958928571428571

In [12]:
data_test = pd.read_csv('test.csv')

In [13]:
data_test['cardio'].shape

(14000,)

[CV 1/5] END ...............var_smoothing=1e-08;, score=0.708 total time=   0.1s
[CV 3/5] END ...............var_smoothing=1e-07;, score=0.680 total time=   0.0s
[CV 1/5] END ...............var_smoothing=1e-09;, score=0.708 total time=   0.0s
[CV 4/5] END ...............var_smoothing=1e-08;, score=0.685 total time=   0.0s
[CV 2/5] END ...............var_smoothing=1e-06;, score=0.686 total time=   0.0s
[CV 3/5] END ...............var_smoothing=1e-09;, score=0.680 total time=   0.1s
[CV 4/5] END ...............var_smoothing=1e-07;, score=0.685 total time=   0.0s
[CV 2/5] END ...............var_smoothing=1e-09;, score=0.686 total time=   0.0s
[CV 5/5] END ...............var_smoothing=1e-08;, score=0.691 total time=   0.0s
[CV 3/5] END ...............var_smoothing=1e-06;, score=0.680 total time=   0.0s
[CV 1/5] END ...............var_smoothing=1e-09;, score=0.708 total time=   0.1s
[CV 5/5] END ...............var_smoothing=1e-07;, score=0.691 total time=   0.0s
[CV 3/5] END ...............

In [14]:
x_test = preprocessor.transform(data_test)
y_test = nb_clf.predict(x_test)

In [15]:
from sklearn.metrics import accuracy_score
accuracy_score(data_test['cardio'], y_test)

0.6006428571428571